# Generating the data
1. Generate the Heterogeneous graph
2. Generate the feature set from the clinical notes.
3. Generate the Labels
4. Generate the k-metapath-based similarity matrices
5. Convert the As to edge-based.

In [1]:
import os
import torch
def save_list_as_pickle(L, given_path, file_name):
    import pickle
    print(f'saving to {given_path}/{file_name}.pkl')
    with open(f'{given_path}/{file_name}.pkl', 'wb') as file:
        pickle.dump(L, file)


num_Diseases    = int(os.getenv('NUM_DISEASES', 203))  
DISEASE_FILE    = os.getenv('DISEASE_FILE', f'DMPLB')  
similarity_type = os.getenv('similarity_type', 'PC')  # options are PC: PathCount, SPS: Symmetric PathSim

num_Sample      = int(os.getenv('num_Sample', 10000))  
r_u_sampling    = os.getenv('r_u_sampling', 'True')  

if r_u_sampling=='True':
    sampling = True
else:
    sampling = False

disease_data_path = '/lustre/home/almusawiaf/PhD_Projects/HGNN_Project/Data'

print(num_Diseases, DISEASE_FILE, similarity_type, num_Sample, sampling)


203 DMPLB PC 10000 True


In [2]:
from module1 import generating_HG as G_class
from module1 import XY_preparation as XY
from module1 import meta_path as MP
# =================================================================================
saving_path = f'{disease_data_path}/{num_Diseases}_Diagnoses/{DISEASE_FILE}/{num_Sample}'

for p in ['HGNN_data', 'clinical_items', 'GMLs', 'OHV', 'PSGs', 'SNFs']:
    os.makedirs(f'{saving_path}/{p}', exist_ok=True)

saving_path = f'{disease_data_path}/{num_Diseases}_Diagnoses/{DISEASE_FILE}/{num_Sample}/HGNN_data'
# =================================================================================
HG_instance = G_class.Generate_HG(sampling = sampling, num_Patients = num_Sample)
HG = HG_instance.HG
G_class.G_statistics(HG)
# =================================================================================
if not sampling:
    num_Sample = len(HG_instance.Patients)
# =================================================================================
XY_inst = XY.XY_preparation(HG)
X = XY_inst.X
Y = XY_inst.Y
# =================================================================================
MP_inst = MP.Meta_path(HG, similarity_type = 'PC', saving_path = saving_path)
# =================================================================================
torch.save(X, f'{saving_path}/X.pt')
torch.save(Y, f'{saving_path}/Y.pt')

save_list_as_pickle(MP_inst.Nodes,   saving_path, 'Nodes')

Loading the dataframes...



We are SAMPLING

Use the patients inside the new DataFrame....


Dropping NaN visits


General Information:
---------------------------
Number of Patients = 10000
Number of Diagnosis = 203
Number of procedures = 88
Number of Medication = 279
Number of Lab tests  = 646
Number of MicroBio   = 82
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Splitting lab tests


/lustre/home/almusawiaf/PhD_Projects/HGNN_Project/version_2/SRC/Data_Generation/module1/generating_HG.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lab_df.loc[:, 'ITEMID_FLAG'] = lab_df['ITEMID'].astype(str) + '_' + lab_df['FLAG'].astype(str)


...

	done...
	Saving...
	Done saving edge_weight39...
Working on 40th file...


	done...
	Saving...
	Done saving edge_weight40...
Working on 41th file...


	done...
	Saving...
	Done saving edge_weight41...
Working on 42th file...


	done...
	Saving...
	Done saving edge_weight42...


saving to /lustre/home/almusawiaf/PhD_Projects/HGNN_Project/Data/203_Diagnoses/DMPLB/10000/HGNN_data/Nodes.pkl
